In [1]:
!pip install cleanvision decord safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 117.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 34.7 MB/s eta 0:00:00


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohamedmustafa/real-life-violence-situations-dataset")

print("Path to dataset files:", path)

100%|██████████| 3.58G/3.58G [00:34<00:00, 110MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mohamedmustafa/real-life-violence-situations-dataset/versions/1


In [3]:
import os
os.listdir(path)

['real life violence situations', 'Real Life Violence Dataset']

In [4]:
import os
import glob
dataset_path = "/root/.cache/kagglehub/datasets/mohamedmustafa/real-life-violence-situations-dataset/versions/1/Real Life Violence Dataset"

violence_videos = glob.glob(os.path.join(dataset_path, "Violence", "*.mp4"))
non_violence_videos = glob.glob(os.path.join(dataset_path, "NonViolence", "*.mp4"))

print("Violence:", len(violence_videos))
print("NonViolence:", len(non_violence_videos))
print("Всего:", len(violence_videos) + len(non_violence_videos))

Violence: 1000
NonViolence: 951
Всего: 1951


In [5]:
import hashlib
import os

def find_duplicates(root_dir):
    hashes = {}
    duplicates = []

    # ВНИМАНИЕ: Проверь имена папок на диске.
    # Судя по твоему скриншоту, они называются 'Violence' и 'NonViolence'
    folders = ['Violence', 'NonViolence']

    for folder in folders:
        path = os.path.join(root_dir, folder)

        # Проверяем, существует ли папка, прежде чем в нее заходить
        if not os.path.exists(path):
            print(f"Предупреждение: Папка не найдена: {path}")
            continue

        for filename in os.listdir(path):
            file_path = os.path.join(path, filename)
            if os.path.isfile(file_path):
                # Считаем хеш файла
                with open(file_path, "rb") as f:
                    file_hash = hashlib.md5(f.read()).hexdigest()

                if file_hash in hashes:
                    duplicates.append(file_path)
                else:
                    hashes[file_hash] = file_path
    return duplicates

dupes = find_duplicates(dataset_path)
print(f"Найдено полных дубликатов: {len(dupes)}")

Найдено полных дубликатов: 14


In [6]:
# Вычитаем дубликаты из основного списка
violence_videos = [path for path in violence_videos if path not in set(dupes)]
non_violence_videos = [path for path in non_violence_videos if path not in set(dupes)]

In [ ]:
import cv2

def is_blurry(video_path, threshold=100.0, sample_frames=5):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Берем несколько кадров из середины видео для проверки
    step = total_frames // (sample_frames + 1)
    blur_scores = []

    for i in range(1, sample_frames + 1):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * step)
        ret, frame = cap.read()
        if not ret:
            continue

        # Переводим в ч/б и считаем дисперсию Лапласиана
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        score = cv2.Laplacian(gray, cv2.CV_64F).var()
        blur_scores.append(score)

    cap.release()

    if not blur_scores:
        return True # Если не смогли прочитать кадры, считаем браком

    avg_score = sum(blur_scores) / len(blur_scores)
    # Если средний скор ниже порога — видео "мыльное"
    return avg_score < threshold, avg_score

# Пример использования:
# blurry, score = is_blurry("video.mp4", threshold=70.0)
# if blurry: print("Видео слишком размытое!")

In [7]:
violence_videos_labels = [1] * len(violence_videos)
non_violence_videos_labels = [0] * len(non_violence_videos)
all_dataset, all_dataset_label = violence_videos + non_violence_videos, violence_videos_labels + non_violence_videos_labels

In [8]:
import cv2
import os

def clean_dataset(paths, labels):
    valid_paths = []
    valid_labels = []

    print(f"Проверка {len(paths)} файлов...")
    for p, l in zip(paths, labels):
        cap = cv2.VideoCapture(p)
        # Проверяем, открывается ли файл и есть ли в нем кадры
        if cap.isOpened() and int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) > 16:
            valid_paths.append(p)
            valid_labels.append(l)
        cap.release()

    print(f"Осталось валидных файлов: {len(valid_paths)}")
    return valid_paths, valid_labels

# Применяем
all_dataset, all_dataset_label = clean_dataset(all_dataset, all_dataset_label)

Проверка 1937 файлов...
Осталось валидных файлов: 1937


In [9]:
import torch
from decord import VideoReader, cpu
import numpy as np

class ViolencDataset(torch.utils.data.Dataset):
    def __init__(self, video_paths, labels, num_frames=16, transform=None):
        self.video_paths = video_paths
        self.labels = labels
        self.num_frames = num_frames
        self.transform = transform

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        vr = VideoReader(self.video_paths[idx], ctx=cpu(0))

        # Выбираем 16 равномерно распределенных кадров
        total_frames = len(vr)
        indices = np.linspace(0, total_frames - 1, self.num_frames, dtype=int)

        frames = vr.get_batch(indices).asnumpy()
        frames = torch.from_numpy(frames).permute(3, 0, 1, 2).float() / 255.0

        if self.transform:
            frames = self.transform(frames)

        return frames, self.labels[idx]

In [10]:
from torchvision import transforms

# Создаем кастомный трансформ для видео, чтобы не путать размерности
class VideoNormalize(object):
    def __init__(self, mean, std):
        self.mean = torch.tensor(mean).view(3, 1, 1, 1)
        self.std = torch.tensor(std).view(3, 1, 1, 1)

    def __call__(self, tensor):
        # tensor: [3, 16, 224, 224]
        return (tensor - self.mean.to(tensor.device)) / self.std.to(tensor.device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    VideoNormalize(mean=[0.4321, 0.3946, 0.3764],
                   std=[0.2280, 0.2214, 0.2170])
])

In [11]:
dataset = ViolencDataset(all_dataset, labels=all_dataset_label, transform=transform)

In [12]:
from torch.utils.data import random_split

In [13]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

In [14]:
import torch
import torch.nn as nn
from torchvision.models.video import r2plus1d_18, R2Plus1D_18_Weights

def get_model():
    weights = R2Plus1D_18_Weights.DEFAULT
    model = r2plus1d_18(weights=weights)

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)

    return model.to("cuda")

In [15]:
from tqdm.auto import tqdm

In [16]:
from torch.cuda.amp import GradScaler, autocast

model = get_model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scaler = GradScaler()

def run_epoch(train_loader, epoch, epochs, model, val_loader, optimizer, criterion, scaler):
    # --- TRAINING PHASE ---
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    # Прогресс-бар для обучения
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training", leave=False)

    for inputs, labels in train_bar:
        inputs, labels = inputs.to("cuda"), labels.to("cuda")

        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Метрики
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()

        # Обновляем инфо в баре
        train_bar.set_postfix(loss=loss.item())

    train_acc = 100. * train_correct / train_total
    avg_train_loss = train_loss / len(train_loader)

    # --- VALIDATION PHASE ---
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation", leave=False)

    with torch.no_grad():
        for inputs, labels in val_bar:
            inputs, labels = inputs.to("cuda"), labels.to("cuda")

            with torch.amp.autocast('cuda'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_acc = 100. * val_correct / val_total
    avg_val_loss = val_loss / len(val_loader)

    print(f"[Epoch {epoch+1}/{epochs}] "
          f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% || "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    return avg_val_loss, val_acc

Downloading: "https://download.pytorch.org/models/r2plus1d_18-91a641e6.pth" to /root/.cache/torch/hub/checkpoints/r2plus1d_18-91a641e6.pth


100%|██████████| 120M/120M [00:00<00:00, 181MB/s]
/tmp/ipython-input-3563408797.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [22]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_set,
    batch_size=8,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)

val_loader = DataLoader(
    val_set,
    batch_size=8,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)


In [23]:
NUM_EPOCHS = 6

for epoch in range(NUM_EPOCHS):
    v_loss, v_acc = run_epoch(train_loader, epoch, NUM_EPOCHS, model, val_loader, optimizer, criterion, scaler)

    if v_acc >= 85.0:
        from safetensors.torch import save_model
        save_model(model, f"model_acc_{v_acc:.2f}.safetensors")
        print(f"--> Target accuracy reached! Model saved.")

Epoch 1/6 - Training:   0%|          | 0/170 [00:00<?, ?it/s]

Epoch 1/6 - Validation:   0%|          | 0/37 [00:00<?, ?it/s]

[Epoch 1/6] Train Loss: 0.2289 | Train Acc: 91.81% || Val Loss: 0.0472 | Val Acc: 98.62%
--> Target accuracy reached! Model saved.


Epoch 2/6 - Training:   0%|          | 0/170 [00:00<?, ?it/s]

Epoch 2/6 - Validation:   0%|          | 0/37 [00:00<?, ?it/s]

...

Epoch 5/6 - Validation:   0%|          | 0/37 [00:00<?, ?it/s]

[Epoch 5/6] Train Loss: 0.0822 | Train Acc: 97.34% || Val Loss: 0.0428 | Val Acc: 98.97%
--> Target accuracy reached! Model saved.


Epoch 6/6 - Training:   0%|          | 0/170 [00:00<?, ?it/s]

Epoch 6/6 - Validation:   0%|          | 0/37 [00:00<?, ?it/s]

[Epoch 6/6] Train Loss: 0.0547 | Train Acc: 98.23% || Val Loss: 0.0446 | Val Acc: 97.59%
--> Target accuracy reached! Model saved.


In [25]:
from safetensors.torch import load_model

In [39]:
def load_my_model(model_path):
    # Создаем архитектуру (должна быть такой же, как при обучении)
    model = r2plus1d_18()
    model.fc = nn.Linear(model.fc.in_features, 2)

    # Загружаем твои веса из safetensors
    load_model(model, model_path)
    model = model.to("cuda").eval()
    return model

def predict_violence(video_path, model, transform):
    # 1. Читаем видео (берем 16 равномерных кадров)
    vr = VideoReader(video_path, ctx=cpu(0))
    indices = np.linspace(0, len(vr) - 1, 16, dtype=int)
    frames = vr.get_batch(indices).asnumpy()

    # 2. Превращаем в тензор [C, T, H, W] -> [3, 16, 224, 224]
    video_tensor = torch.from_numpy(frames).permute(3, 0, 1, 2).float() / 255.0

    # 3. Применяем нормализацию (используй тот же VideoNormalize, что был в обучении)
    video_tensor = transform(video_tensor).unsqueeze(0).to("cuda")

    # 4. Прогон через нейронку
    with torch.no_grad():
        with torch.amp.autocast('cuda'):
            outputs = model(video_tensor)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            confidence, prediction = torch.max(probs, 1)

    classes = ["Safe (Non-Violence)", "Violence Detected! ⚠️"]
    return classes[prediction.item()], confidence.item()

# --- ПРИМЕНЕНИЕ ---
# Укажи путь к своему лучшему файлу
MY_MODEL_PATH = glob.glob(os.path.join("/content", "*.safetensors"))

In [42]:
test_videos = glob.glob(os.path.join("/content", "*.mp4"))

for model_path in MY_MODEL_PATH:
    model = load_my_model(model_path)
    print("============= MODEL =============")
    print(model_path)
    for test_video in test_videos:
        result, score = predict_violence(test_video, model, transform)

        print(f"Video path: {test_video}")
        print(f"Результат анализа: {result}")
        print(f"Уверенность модели: {score*100:.2f}%")
        print(f"\n\n")

============= MODEL =============
/content/model_acc_98.28.safetensors
Video path: /content/video_2026-02-23_15-21-00.mp4
Результат анализа: Safe (Non-Violence)
Уверенность модели: 90.11%



Video path: /content/video_2026-02-23_15-21-54.mp4
Результат анализа: Violence Detected! ⚠️
Уверенность модели: 93.72%



Video path: /content/video_2026-02-23_15-21-15.mp4
Результат анализа: Safe (Non-Violence)
Уверенность модели: 83.65%



Video path: /content/video_2026-02-23_13-24-24.mp4
Результат анализа: Violence Detected! ⚠️
Уверенность модели: 54.02%



Video path: /content/video_2026-02-23_15-20-42.mp4
Результат анализа: Violence Detected! ⚠️
Уверенность модели: 99.60%



============= MODEL =============
/content/model_acc_97.93.safetensors
Video path: /content/video_2026-02-23_15-21-00.mp4
Результат анализа: Safe (Non-Violence)
Уверенность модели: 98.21%



Video path: /content/video_2026-02-23_15-21-54.mp4
Результат анализа: Violence Detected! ⚠️
Уверенность модели: 93.77%



Video path